Job Application Assistant

Overview

This tool helps with job applications by comparing a resume to a job description. It then creates a personalized cover letter, points out any skill gaps, and generates practice interview questions.

Input: Resume (PDF/as location ) + Job description (as text)

Output: PDF report with tailored cover letter, skill gap analysis, interview preparation questions

In [ ]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_openai langchain_community langchain_core tavily-pytJob Application Assistant


Setup

In [ ]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [ ]:

_set_env("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "job-application-assistant"

In [ ]:

_set_env("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "job-application-assistant"

PDF Resume Reader

Upload and extract text from PDF resume files.

In [ ]:
from pypdf import PdfReader

def read_resume_pdf(pdf_path: str) -> str:
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text.strip()

Data Models (Structured Output)

We define Pydantic models to ensure structured, validated output from the LLM.

In [ ]:
from typing import List, Optional
from typing_extensions import TypedDict
from pydantic import BaseModel, Field

class Experience(BaseModel):
    company: str = Field(description="Company name")
    role: str = Field(description="Job title/role")
    duration: str = Field(description="Duration of employment")
    highlights: List[str] = Field(description="Key achievements and responsibilities")

class Education(BaseModel):
    institution: str = Field(description="School/University name")
    degree: str = Field(description="Degree obtained")
    year: str = Field(description="Year of graduation")
    gpa: Optional[str] = Field(default=None, description="GPA if mentioned")

class ParsedResume(BaseModel):
    name: str = Field(description="Candidate's full name")
    email: Optional[str] = Field(default=None, description="Email address")
    phone: Optional[str] = Field(default=None, description="Phone number")
    summary: str = Field(description="Professional summary")
    skills: List[str] = Field(description="Technical and soft skills")
    experience: List[Experience] = Field(description="Work experience")
    education: List[Education] = Field(description="Educational background")
    certifications: List[str] = Field(default=[], description="Certifications")

class JobRequirements(BaseModel):
    title: str = Field(description="Job title")
    company: str = Field(description="Company name")
    required_skills: List[str] = Field(description="Required technical skills")
    preferred_skills: List[str] = Field(description="Nice-to-have skills")
    responsibilities: List[str] = Field(description="Key job responsibilities")
    qualifications: List[str] = Field(description="Required qualifications")
    experience_years: Optional[str] = Field(default=None, description="Years of experience required")

class SkillMatch(BaseModel):
    skill: str = Field(description="The skill being analyzed")
    status: str = Field(description="'matched', 'partial', or 'missing'")
    evidence: Optional[str] = Field(default=None, description="Evidence from resume if matched")

class SkillGapAnalysis(BaseModel):
    match_score: int = Field(description="Overall match percentage 0-100")
    matched_skills: List[SkillMatch] = Field(description="Skills that match")
    partial_matches: List[SkillMatch] = Field(description="Transferable/related skills")
    missing_skills: List[SkillMatch] = Field(description="Skills to develop")
    recommendations: List[str] = Field(description="Suggestions to bridge gaps")

class CompanyResearch(BaseModel):
    company_name: str = Field(description="Company name")
    description: str = Field(description="What the company does")
    culture: str = Field(description="Company culture and values")
    recent_news: List[str] = Field(description="Recent news or achievements")
    interview_tips: List[str] = Field(description="Tips for interviewing at this company")

class CoverLetter(BaseModel):
    greeting: str = Field(description="Opening greeting")
    opening_paragraph: str = Field(description="Hook and introduction")
    body_paragraphs: List[str] = Field(description="Main content paragraphs")
    closing_paragraph: str = Field(description="Call to action and closing")
    signature: str = Field(description="Sign-off and name")
    
    def to_text(self) -> str:
        body = "\n\n".join(self.body_paragraphs)
        return f"{self.greeting}\n\n{self.opening_paragraph}\n\n{body}\n\n{self.closing_paragraph}\n\n{self.signature}"

class InterviewQuestion(BaseModel):
    question: str = Field(description="The interview question")
    category: str = Field(description="Category: technical, behavioral, situational, company-specific")
    suggested_answer: str = Field(description="Suggested answer approach based on resume")
    tips: str = Field(description="Tips for answering")

class InterviewPrep(BaseModel):
    technical_questions: List[InterviewQuestion] = Field(description="Technical questions")
    behavioral_questions: List[InterviewQuestion] = Field(description="Behavioral questions")
    company_specific: List[InterviewQuestion] = Field(description="Company-specific questions")
    questions_to_ask: List[str] = Field(description="Questions candidate should ask")

LangGraph State

The state holds all information as it flows through our multi-agent system.

In [ ]:
class ApplicationState(TypedDict):
    resume_text: str
    job_description: str
    parsed_resume: Optional[ParsedResume]
    job_requirements: Optional[JobRequirements]
    company_research: Optional[CompanyResearch]
    skill_gap: Optional[SkillGapAnalysis]
    cover_letter: Optional[CoverLetter]
    interview_prep: Optional[InterviewPrep]
    human_feedback: Optional[str]

Node 1: Parse Resume

Extract structured information from the resume using prompting and structured output.

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage

resume_parser_prompt = """You are an expert resume parser. Analyze the provided resume and extract all relevant information.

Be thorough in extracting:
- All skills mentioned (technical, soft skills, tools, frameworks)
- Complete work history with achievements
- Educational background
- Certifications and additional qualifications

If information is not explicitly stated, make reasonable inferences but note them."""

def parse_resume(state: ApplicationState) -> dict:
    structured_llm = llm.with_structured_output(ParsedResume)
    
    result = structured_llm.invoke([
        SystemMessage(content=resume_parser_prompt),
        HumanMessage(content=f"Parse this resume:\n\n{state['resume_text']}")
    ])
    
    return {"parsed_resume": result}

Node 2: Analyze Job Description

Extract requirements and expectations from the job posting.

In [ ]:
job_analyzer_prompt = """You are an expert job description analyzer. Extract all requirements from the job posting.

Focus on:
- Required vs preferred skills (be specific)
- Key responsibilities and expectations
- Qualifications and experience requirements
- Any hints about company culture or team dynamics

Distinguish between must-have and nice-to-have requirements."""

def analyze_job(state: ApplicationState) -> dict:
    structured_llm = llm.with_structured_output(JobRequirements)
    
    result = structured_llm.invoke([
        SystemMessage(content=job_analyzer_prompt),
        HumanMessage(content=f"Analyze this job description:\n\n{state['job_description']}")
    ])
    
    return {"job_requirements": result}

Node 3: Research Company (Tool Calling)

Use Tavily web search to gather company information.

In [ ]:
from langchain_tavily import TavilySearch

tavily_search = TavilySearch(max_results=5)

company_research_prompt = """You are a company research specialist. Based on the search results, compile useful information for a job applicant.

Focus on:
- What the company does and their main products/services
- Company culture and values
- Recent news, achievements, or challenges
- Tips for interviewing at this company

Search Results:
{search_results}
"""

def research_company(state: ApplicationState) -> dict:
    company_name = state["job_requirements"].company
    
    search_queries = [
        f"{company_name} company culture values",
        f"{company_name} recent news 2024",
        f"{company_name} interview tips glassdoor"
    ]
    
    all_results = []
    for query in search_queries:
        results = tavily_search.invoke(query)
        all_results.append(f"Query: {query}\nResults: {results}")
    
    search_results = "\n\n".join(all_results)
    
    structured_llm = llm.with_structured_output(CompanyResearch)
    
    result = structured_llm.invoke([
        SystemMessage(content=company_research_prompt.format(search_results=search_results)),
        HumanMessage(content=f"Compile research about {company_name} for a job applicant.")
    ])
    
    return {"company_research": result}

Node 4: Skill Gap Analysis (Semantic Search)

Compare resume skills with job requirements using semantic matching.

In [ ]:
skill_gap_prompt = """You are a career advisor performing skill gap analysis.

Compare the candidate's skills and experience against the job requirements.

CANDIDATE PROFILE:
Skills: {skills}
Experience: {experience}

JOB REQUIREMENTS:
Required Skills: {required_skills}
Preferred Skills: {preferred_skills}
Qualifications: {qualifications}

Perform SEMANTIC matching - consider:
- Exact matches (Python = Python)
- Related technologies (React experience helps with Vue.js)
- Transferable skills (leadership in one context applies to another)
- Experience level matching

Provide actionable recommendations to bridge any gaps."""

def analyze_skill_gap(state: ApplicationState) -> dict:
    resume = state["parsed_resume"]
    job = state["job_requirements"]
    
    experience_text = "\n".join([
        f"- {exp.role} at {exp.company}: {', '.join(exp.highlights[:3])}"
        for exp in resume.experience
    ])
    
    structured_llm = llm.with_structured_output(SkillGapAnalysis)
    
    result = structured_llm.invoke([
        SystemMessage(content=skill_gap_prompt.format(
            skills=", ".join(resume.skills),
            experience=experience_text,
            required_skills=", ".join(job.required_skills),
            preferred_skills=", ".join(job.preferred_skills),
            qualifications=", ".join(job.qualifications)
        )),
        HumanMessage(content="Analyze the skill gap and provide recommendations.")
    ])
    
    return {"skill_gap": result}

Node 5: Generate Cover Letter (RAG)

Use resume content as context (RAG) to generate a tailored cover letter.

In [ ]:
cover_letter_prompt = """You are an expert cover letter writer. Create a compelling, personalized cover letter.

CANDIDATE INFORMATION (use as context for personalization):
Name: {name}
Summary: {summary}
Key Skills: {skills}
Relevant Experience:
{experience}

JOB DETAILS:
Position: {job_title} at {company}
Key Requirements: {requirements}

COMPANY INSIGHTS:
{company_info}

SKILL MATCH HIGHLIGHTS:
{skill_highlights}

Write a cover letter that:
1. Opens with a compelling hook related to the company or role
2. Highlights 2-3 specific achievements that match job requirements
3. Shows knowledge of the company (use research insights)
4. Addresses any skill gaps positively (growth mindset)
5. Ends with a confident call to action

Be specific, not generic. Use concrete examples from the resume."""

def generate_cover_letter(state: ApplicationState) -> dict:
    resume = state["parsed_resume"]
    job = state["job_requirements"]
    company = state["company_research"]
    skill_gap = state["skill_gap"]
    
    experience_text = "\n".join([
        f"- {exp.role} at {exp.company} ({exp.duration}):\n  " + "\n  ".join(exp.highlights)
        for exp in resume.experience
    ])
    
    skill_highlights = "\n".join([
        f"- {match.skill}: {match.evidence}"
        for match in skill_gap.matched_skills[:5]
        if match.evidence
    ])
    
    structured_llm = llm.with_structured_output(CoverLetter)
    
    result = structured_llm.invoke([
        SystemMessage(content=cover_letter_prompt.format(
            name=resume.name,
            summary=resume.summary,
            skills=", ".join(resume.skills[:10]),
            experience=experience_text,
            job_title=job.title,
            company=job.company,
            requirements=", ".join(job.required_skills[:5]),
            company_info=f"{company.description}\nCulture: {company.culture}",
            skill_highlights=skill_highlights
        )),
        HumanMessage(content="Write the cover letter.")
    ])
    
    return {"cover_letter": result}

Node 6: Generate Interview Questions

Prepare interview questions based on skill gaps and job requirements.

In [ ]:
interview_prep_prompt = """You are an interview coach preparing a candidate for their job interview.

CANDIDATE PROFILE:
Experience: {experience}
Skills: {skills}

JOB REQUIREMENTS:
Role: {job_title} at {company}
Required Skills: {required_skills}
Responsibilities: {responsibilities}

SKILL GAP ANALYSIS:
Match Score: {match_score}%
Areas to Address: {skill_gaps}

COMPANY INSIGHTS:
{company_insights}

Generate interview preparation including:
1. Technical questions they're likely to face (based on required skills)
2. Behavioral questions (STAR method answers using their experience)
3. Company-specific questions (based on company research)
4. Smart questions the candidate should ask the interviewer

For each question, provide a suggested answer approach based on their resume."""

def generate_interview_prep(state: ApplicationState) -> dict:
    resume = state["parsed_resume"]
    job = state["job_requirements"]
    company = state["company_research"]
    skill_gap = state["skill_gap"]
    
    experience_text = "; ".join([
        f"{exp.role} at {exp.company}"
        for exp in resume.experience
    ])
    
    skill_gaps = ", ".join([
        gap.skill for gap in skill_gap.missing_skills[:5]
    ])
    
    structured_llm = llm.with_structured_output(InterviewPrep)
    
    result = structured_llm.invoke([
        SystemMessage(content=interview_prep_prompt.format(
            experience=experience_text,
            skills=", ".join(resume.skills[:10]),
            job_title=job.title,
            company=job.company,
            required_skills=", ".join(job.required_skills),
            responsibilities="; ".join(job.responsibilities[:5]),
            match_score=skill_gap.match_score,
            skill_gaps=skill_gaps if skill_gaps else "No major gaps",
            company_insights="\n".join(company.interview_tips)
        )),
        HumanMessage(content="Generate comprehensive interview preparation.")
    ])
    
    return {"interview_prep": result}

Human-in-the-Loop Node

Allow human review before generating final outputs.

In [ ]:
def human_review(state: ApplicationState) -> dict:
    pass

def should_regenerate(state: ApplicationState) -> str:
    feedback = state.get("human_feedback")
    if feedback:
        return "generate_cover_letter"
    return "end"

Build the LangGraph

Assemble all nodes into a complete workflow with state, edges, and human-in-the-loop.

In [ ]:
from IPython.display import Image, display
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver

builder = StateGraph(ApplicationState)

builder.add_node("parse_resume", parse_resume)
builder.add_node("analyze_job", analyze_job)
builder.add_node("research_company", research_company)
builder.add_node("analyze_skill_gap", analyze_skill_gap)
builder.add_node("generate_cover_letter", generate_cover_letter)
builder.add_node("generate_interview_prep", generate_interview_prep)
builder.add_node("human_review", human_review)

builder.add_edge(START, "parse_resume")
builder.add_edge("parse_resume", "analyze_job")
builder.add_edge("analyze_job", "research_company")
builder.add_edge("research_company", "analyze_skill_gap")
builder.add_edge("analyze_skill_gap", "generate_cover_letter")
builder.add_edge("generate_cover_letter", "generate_interview_prep")
builder.add_edge("generate_interview_prep", "human_review")

builder.add_conditional_edges(
    "human_review",
    should_regenerate,
    {"generate_cover_letter": "generate_cover_letter", "end": END}
)

memory = MemorySaver()
graph = builder.compile(interrupt_before=["human_review"], checkpointer=memory)

display(Image(graph.get_graph(xray=1).draw_mermaid_png()))

Load Resume and Job Description

Upload your resume PDF and paste the job description.

In [ ]:
RESUME_PDF_PATH = "/home/jai/study_material/r_c/ResumeJaideepSinghB.pdf"

try:
    resume_text = read_resume_pdf(RESUME_PDF_PATH)
    print(f"Successfully loaded resume from: {RESUME_PDF_PATH}")
    print(f"   Extracted {len(resume_text)} characters")
    print("\n--- Preview (first 500 chars) ---")
    print(resume_text[:500])
except FileNotFoundError:
    print(f"Resume file not found: {RESUME_PDF_PATH}")
    print("   Please update RESUME_PDF_PATH with the correct path to your resume PDF")
    resume_text = """ """

Xanadu Realty is the company where I am placed, So I though why don't I add it's job description and test this tool on a real life scenario.

In [ ]:
job_description = """
Full-Stack Engineer at Xanadu Realty

About the Role:
We're looking for a Full-Stack Engineer to join our Cloud Platform team.
You'll work on building next-generation developer tools used by millions.

Requirements:
- Fresher student with high CGPA / Academic Background.
- Strong proficiency in Python and JavaScript
- Experience with React, Node.js, Express.js and SQL/NoSQL Databases.
- Cloud platform experience Azure
- Experience with Version Control (Git) and CI/CD.
- Strong communication and collaboration skills

Nice to Have:
- Experience with Cyber Security.
- Contributions to open-source projects
- Experience with AI
- Technical leadership experience

Responsibilities:
- Design and implement scalable full-stack applications
- Collaborate with product managers and designers
- Mentor junior engineers and conduct code reviews
- Drive technical decisions and architecture discussions
- Participate in on-call rotation

Benefits:
- Competitive salary and equity
- Health, dental, and vision insurance
- Flexible work arrangements
"""

print("Job Description loaded!")
print(f"Position: Full-Stack Engineer at Xanadu Realty")

Run the Assistant

In [ ]:
thread = {"configurable": {"thread_id": "1"}}

initial_state = {
    "resume_text": resume_text,
    "job_description": job_description
}

print("Running Job Application Assistant...\n")
print("=" * 60)

for event in graph.stream(initial_state, thread, stream_mode="values"):
    if event.get("parsed_resume"):
        print(f"✓ Resume parsed: {event['parsed_resume'].name}")
        print(f"  Skills found: {len(event['parsed_resume'].skills)}")
    
    if event.get("job_requirements"):
        print(f"✓ Job analyzed: {event['job_requirements'].title} at {event['job_requirements'].company}")
    
    if event.get("company_research"):
        print(f"✓ Company researched: {event['company_research'].company_name}")
    
    if event.get("skill_gap"):
        print(f"✓ Skill gap analysis: {event['skill_gap'].match_score}% match")
    
    if event.get("cover_letter"):
        print("✓ Cover letter generated")
    
    if event.get("interview_prep"):
        print("✓ Interview prep generated")

print("\n" + "=" * 60)

In [ ]:
state = graph.get_state(thread)
print(f"Paused at: {state.next}")

View Results

In [ ]:
values = state.values
skill_gap = values.get("skill_gap")

if skill_gap:
    print("=" * 60)
    print("SKILL GAP ANALYSIS")
    print("=" * 60)
    print(f"\nOverall Match Score: {skill_gap.match_score}%\n")
    
    print("✅ MATCHED SKILLS:")
    for skill in skill_gap.matched_skills[:5]:
        print(f"  - {skill.skill}")
        if skill.evidence:
            print(f"    Evidence: {skill.evidence}")
    
    print("\n🔄 PARTIAL MATCHES (Transferable):")
    for skill in skill_gap.partial_matches[:3]:
        print(f"  - {skill.skill}")
    
    print("\n❌ SKILLS TO DEVELOP:")
    for skill in skill_gap.missing_skills[:3]:
        print(f"  - {skill.skill}")
    
    print("\n💡 RECOMMENDATIONS:")
    for rec in skill_gap.recommendations:
        print(f"  • {rec}")

In [ ]:
cover_letter = values.get("cover_letter")

if cover_letter:
    print("=" * 60)
    print("GENERATED COVER LETTER")
    print("=" * 60)
    print(cover_letter.to_text())

In [ ]:
interview_prep = values.get("interview_prep")

if interview_prep:
    print("=" * 60)
    print("INTERVIEW PREPARATION")
    print("=" * 60)
    
    print("\n📚 TECHNICAL QUESTIONS:")
    for i, q in enumerate(interview_prep.technical_questions[:3], 1):
        print(f"\n{i}. {q.question}")
        print(f"   Suggested approach: {q.suggested_answer}")
        print(f"   Tip: {q.tips}")
    
    print("\n🤝 BEHAVIORAL QUESTIONS:")
    for i, q in enumerate(interview_prep.behavioral_questions[:3], 1):
        print(f"\n{i}. {q.question}")
        print(f"   Suggested approach: {q.suggested_answer}")
    
    print("\n🏢 COMPANY-SPECIFIC QUESTIONS:")
    for i, q in enumerate(interview_prep.company_specific[:2], 1):
        print(f"\n{i}. {q.question}")
        print(f"   Tip: {q.tips}")
    
    print("\n❓ QUESTIONS TO ASK THE INTERVIEWER:")
    for q in interview_prep.questions_to_ask:
        print(f"  • {q}")

In [ ]:
graph.update_state(thread, {"human_feedback": None}, as_node="human_review")

for event in graph.stream(None, thread, stream_mode="updates"):
    node_name = next(iter(event.keys()))
    print(f"Completed: {node_name}")

print("\n✅ Job Application Assistant completed!")

Company Research Results

In [ ]:
company = values.get("company_research")

if company:
    print("=" * 60)
    print(f"COMPANY RESEARCH: {company.company_name}")
    print("=" * 60)
    
    print(f"\n📝 About: {company.description}")
    print(f"\n🎯 Culture: {company.culture}")
    
    print("\n📰 Recent News:")
    for news in company.recent_news:
        print(f"  • {news}")
    
    print("\n💡 Interview Tips:")
    for tip in company.interview_tips:
        print(f"  • {tip}")

Export Results to PDF Report

Generate a professional PDF report with all the results.

In [ ]:
from fpdf import FPDF
import textwrap

class PDFReport(FPDF):
    def header(self):
        self.set_font('Helvetica', 'B', 16)
        self.cell(0, 10, 'Job Application Assistant Report', 0, 1, 'C')
        self.ln(5)
    
    def footer(self):
        self.set_y(-15)
        self.set_font('Helvetica', 'I', 8)
        self.cell(0, 10, f'Page {self.page_no()}', 0, 0, 'C')
    
    def chapter_title(self, title):
        self.set_font('Helvetica', 'B', 14)
        self.set_fill_color(230, 230, 230)
        self.cell(0, 10, title, 0, 1, 'L', True)
        self.ln(3)
    
    def section_title(self, title):
        self.set_font('Helvetica', 'B', 11)
        self.cell(0, 8, title, 0, 1)
    
    def body_text(self, text):
        self.set_font('Helvetica', '', 10)
        wrapped = textwrap.fill(text, width=95)
        self.multi_cell(0, 5, wrapped)
        self.ln(2)
    
    def bullet_point(self, text):
        self.set_font('Helvetica', '', 10)
        wrapped = textwrap.fill(text, width=90)
        self.cell(5, 5, chr(149))
        self.multi_cell(0, 5, wrapped)

def generate_pdf_report(values, output_path="job_application_report.pdf"):
    pdf = PDFReport()
    pdf.add_page()
    
    parsed_resume = values.get("parsed_resume")
    job_requirements = values.get("job_requirements")
    skill_gap = values.get("skill_gap")
    cover_letter = values.get("cover_letter")
    interview_prep = values.get("interview_prep")
    company_research = values.get("company_research")
    
    if parsed_resume and job_requirements:
        pdf.chapter_title("Application Summary")
        pdf.body_text(f"Candidate: {parsed_resume.name}")
        pdf.body_text(f"Position: {job_requirements.title} at {job_requirements.company}")
        if skill_gap:
            pdf.body_text(f"Match Score: {skill_gap.match_score}%")
        pdf.ln(5)
    
    if skill_gap:
        pdf.chapter_title("Skill Gap Analysis")
        
        pdf.section_title("Matched Skills")
        for skill in skill_gap.matched_skills[:5]:
            evidence = f" - {skill.evidence}" if skill.evidence else ""
            pdf.bullet_point(f"{skill.skill}{evidence}")
        pdf.ln(3)
        
        pdf.section_title("Transferable Skills")
        for skill in skill_gap.partial_matches[:3]:
            pdf.bullet_point(skill.skill)
        pdf.ln(3)
        
        pdf.section_title("Skills to Develop")
        for skill in skill_gap.missing_skills[:3]:
            pdf.bullet_point(skill.skill)
        pdf.ln(3)
        
        pdf.section_title("Recommendations")
        for rec in skill_gap.recommendations:
            pdf.bullet_point(rec)
        pdf.ln(5)
    
    if cover_letter:
        pdf.add_page()
        pdf.chapter_title("Cover Letter")
        pdf.body_text(cover_letter.to_text())
        pdf.ln(5)
    
    if interview_prep:
        pdf.add_page()
        pdf.chapter_title("Interview Preparation")
        
        pdf.section_title("Technical Questions")
        for i, q in enumerate(interview_prep.technical_questions[:3], 1):
            pdf.set_font('Helvetica', 'B', 10)
            pdf.multi_cell(0, 5, f"{i}. {q.question}")
            pdf.set_font('Helvetica', '', 9)
            pdf.multi_cell(0, 5, f"   Answer approach: {q.suggested_answer}")
            pdf.multi_cell(0, 5, f"   Tip: {q.tips}")
            pdf.ln(2)
        pdf.ln(3)
        
        pdf.section_title("Behavioral Questions")
        for i, q in enumerate(interview_prep.behavioral_questions[:3], 1):
            pdf.set_font('Helvetica', 'B', 10)
            pdf.multi_cell(0, 5, f"{i}. {q.question}")
            pdf.set_font('Helvetica', '', 9)
            pdf.multi_cell(0, 5, f"   Answer approach: {q.suggested_answer}")
            pdf.ln(2)
        pdf.ln(3)
        
        pdf.section_title("Questions to Ask the Interviewer")
        for q in interview_prep.questions_to_ask:
            pdf.bullet_point(q)
        pdf.ln(5)
    
    if company_research:
        pdf.add_page()
        pdf.chapter_title(f"Company Research: {company_research.company_name}")
        
        pdf.section_title("About the Company")
        pdf.body_text(company_research.description)
        
        pdf.section_title("Company Culture")
        pdf.body_text(company_research.culture)
        
        pdf.section_title("Recent News")
        for news in company_research.recent_news:
            pdf.bullet_point(news)
        pdf.ln(3)
        
        pdf.section_title("Interview Tips")
        for tip in company_research.interview_tips:
            pdf.bullet_point(tip)
    
    pdf.output(output_path)
    return output_path

print("PDF Report generator ready!")

In [ ]:
output_file = generate_pdf_report(values, "job_application_report.pdf")
print(f"✅ PDF Report saved to: {output_file}")
print("\nThe report includes:")
print("  • Application Summary")
print("  • Skill Gap Analysis")
print("  • Cover Letter")
print("  • Interview Preparation")
print("  • Company Research")